#Project 2: Generate Parallel Databases

Create a list of every parallel database to the one currently contained in the "db" variable. Create a function which both:

* creates the initial database (db) 
* creates all parallel databases

In [0]:
import torch

In [0]:
def create_parallel_db(db, index):

    return torch.cat((db[0:index], 
                      db[index+1:]))

In [4]:
num_els = 5000
db = torch.rand(num_els) > 0.5
db

tensor([1, 1, 0,  ..., 0, 1, 0], dtype=torch.uint8)

In [5]:
create_parallel_db(db, 47)

tensor([1, 1, 0,  ..., 0, 1, 0], dtype=torch.uint8)

In [0]:
def create_parallel_dbs(db):

    parallel_dbs = list()

    for i in range(len(db)):
        pdb = create_parallel_db(db, i)
        parallel_dbs.append(pdb)
    
    return parallel_dbs

In [0]:
pdbs = create_parallel_dbs(db)

In [0]:
def create_db_and_parallel_dbs(num_els):
    
    db = torch.rand(num_els) > 0.5
    pdbs = create_parallel_dbs(db)
    
    return db, pdbs

In [0]:
db, pdbs = create_db_and_parallel_dbs(40)

## Evaluate The Differential Privacy of a Function
Evaluate how much privacy is leaked by measuring the maximum amount the query changes when someone is removed (maximum over all possible people who could be removed). Measure the sensitivity of sum() function

In [0]:
db, pdbs = create_db_and_parallel_dbs(5000)

In [0]:
def query_sum(db):
    return db.sum()

In [0]:
entire_db_result = query_sum(db)

In [0]:
query_sensitivity = 0
for pdb in pdbs:
    pdb_result = query_sum(pdb)
    
    db_difference = torch.abs(pdb_result - entire_db_result)
    
    if(db_difference > query_sensitivity):
        query_sensitivity = db_difference

In [20]:
query_sensitivity

tensor(1)

#Project 3 - Evaluate the Privacy of a Function
Implement function to compute the "sensitivity"of the mean() function we use for the query when measuring the difference between each parallel db's query result and the query result for the entire database (max value).

In [0]:
def query_mean(db):
    return db.float().mean()

In [0]:
def f_sensitivity(f, n_els=1000):

    db, pdbs = create_db_and_parallel_dbs(n_els)
    
    entire_db_result = f(db)
    
    max_difference = 0
    for pdb in pdbs:
        pdb_result = f(pdb)

        db_difference = torch.abs(pdb_result - entire_db_result)

        if(db_difference > max_difference):
            max_difference = db_difference
            
    return max_difference

In [33]:
f_sensitivity(query_mean)

tensor(0.0005)

In [0]:
db, pdbs = create_db_and_parallel_dbs(20)

In [35]:
db

tensor([1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1],
       dtype=torch.uint8)

#Project: Calculate L1 Sensitivity For Threshold
Define function which calculates the sensitivty for a "threshold"  by computing the sum over the databases and return whether that sum is greater than a certain threshold

In [0]:
def query_sum_threshold(db, threshold=5):
    return (db.sum() > threshold).float()

In [37]:
for i in range(10):
    sens_f = f_sensitivity(query_sum_threshold, n_els=10)
    print(sens_f)

tensor(1.)
0
0
0
0
tensor(1.)
0
tensor(1.)
0
0


#Project: Perform a Differencing Attack on Row 10
Create a database and then two different sum queries to expose the value of the person represented by row 10 in the database

In [0]:
db, _ = create_db_and_parallel_dbs(100)

In [0]:
pdb = create_parallel_db(db, index=10)

In [44]:
db[10]

tensor(1, dtype=torch.uint8)

In [45]:
sum(db)

tensor(51, dtype=torch.uint8)

In [47]:
# Differencing attack using sum()

sum(db) - sum(pdb)

tensor(1, dtype=torch.uint8)

In [48]:
# Differencing attack using mean()

(sum(db).float() / len(db)) -  (sum(pdb).float() / len(pdb))

tensor(0.0049)

In [51]:
# Differencing attack using sum() nad threshold

(sum(db).float() > 49) - (sum(pdb).float()  > 49)

tensor(0, dtype=torch.uint8)